In [1]:
import numpy as np
import time
from collections import deque
from random import randint

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import Legend, LegendItem
from bokeh.models import Range1d
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

# Serial data readout

Stateful generator for parsing the serial feed from the sensor, rebooting it at start up.

In [2]:
import enum
from typing import Any, Iterator


def parse_measurement_line(line: str) -> list[float]:
    return list((float(e) for e in line.split(',')))


# TODO: replace Any with a proper type hint
def console_parser(serial_feed: Iterator[str]) -> Iterator[Any]:
    class State(enum.Enum):
        BOOTING = enum.auto()
        WAITING_FOR_FIRST = enum.auto()
        SAMPLING = enum.auto()

    state = State.BOOTING
    
    for msg in serial_feed:
        next_state = state
        
        if state == State.BOOTING:
            if 'WHO_AM_I: 0x' in msg:
                next_state = State.WAITING_FOR_FIRST
                
        elif state == State.WAITING_FOR_FIRST:
            try:
                yield parse_measurement_line(msg)
                next_state = State.SAMPLING
            except Exception:
                pass

        elif state == State.SAMPLING:
            if msg.startswith("DBG:"):
                # print(msg)
                8==0
            else:
                try:
                    yield parse_measurement_line(msg)
                except Exception:
                    next_state = State.BOOTING

        if state != next_state:
            print(f">>>Change state to {next_state.name}")
        state = next_state

Generator that wraps the serial port as a stream of lines

In [3]:
import time
import serial


def lines_from_serial(port="/dev/ttyACM0",
                      baudrate=115200) -> Iterator[str]:
    with serial.Serial(port=port,
                       baudrate=baudrate,
                       dsrdtr=False) as sp:
        # Pulse DTR for resetting
        sp.setDTR(True)
        time.sleep(0.5)
        sp.setDTR(False)

        while True:
            read_in: bytes = sp.readline()
            line = read_in.decode().rstrip()
            yield line

Chain both generators:

In [5]:
lines = lines_from_serial()
for i, meas in enumerate(console_parser(lines)):
    print(time.time(), meas)
    if i == 10:
        break

>>>Change state to WAITING_FOR_FIRST
1720258403.221895 [1.051, -0.076, 1.523]
>>>Change state to SAMPLING
1720258403.230414 [1.056, -0.117, 1.499]
1720258403.2405684 [1.063, -0.115, 1.552]
1720258403.250085 [1.056, -0.146, 1.504]
1720258403.2601912 [1.065, -0.11, 1.552]
1720258403.2705328 [1.031, -0.086, 1.6]
1720258403.2809718 [1.06, -0.096, 1.459]
1720258403.2903855 [1.047, -0.122, 1.595]
1720258403.3011992 [1.064, -0.096, 1.542]
1720258403.3106902 [1.057, -0.091, 1.514]
1720258403.3206034 [1.05, -0.125, 1.552]


## Receive-side sample pre-processsing

Various pre-processing functions, including the trivial identity one

In [6]:
def identity(meas: list[float]) -> list[float]:
    return meas

In [7]:
# The offsets are calculated on the "Some tests" section below
def deoffset(meas: list[float]) -> list[float]:
    return [meas[0] - off_ax, meas[1] - off_ay, meas[2] - off_az]

In [8]:
import enum
from typing import Optional, Callable


class MotionState(enum.Enum):
    AT_REST = enum.auto()
    MOVING = enum.auto()


class MotionObserver:
    
    def __init__(self, mag_reset_thd: float, window: int, state_cb: Optional[Callable]=None):
        self.mag_reset_threshold = mag_reset_thd
        self.state_cb = state_cb
        
        self.buffer = deque(maxlen=window)
        self.state = MotionState.AT_REST

    def __call__(self, new: list[float]) -> float:
        vec = np.asarray(new)
        self.buffer.append(np.linalg.norm(vec))
        figure = np.percentile(np.asarray(self.buffer), 75)
        self.magnitude_reset_criterion(figure)
        return figure

    def magnitude_reset_criterion(self, magnitude: float) -> None:
        next_state = self.state
        if self.state == MotionState.AT_REST and magnitude > self.mag_reset_threshold:
            next_state = MotionState.MOVING
        elif self.state == MotionState.MOVING and magnitude < self.mag_reset_threshold:
            next_state = MotionState.AT_REST

        if next_state != self.state and self.state_cb:
            self.state_cb(next_state.name.replace("_", " "))
        self.state = next_state

In [9]:
class OffsetState(enum.Enum):
    AT_REST_ACCUMULATING = enum.auto()
    AT_REST_IDLE = enum.auto()
    MOVING = enum.auto()


class DynamicOffsetCompensator:
    
    def __init__(self, mag_reset_thd: float, window: int, state_cb: Optional[Callable]):
        self.mag_reset_threshold = mag_reset_thd
        self.state_cb = state_cb

        self.offset = np.r_[0, 0, 0]
        self.buffer = deque(maxlen=window)
        self.state = OffsetState.AT_REST_ACCUMULATING

    def __call__(self, new: list[float]) -> list[float]:
        self.buffer.append(new)
        vecs = np.asarray(self.buffer)
        magnitudes = np.linalg.norm(vecs, axis=1)
        figure = np.percentile(magnitudes, 75)
        self.magnitude_reset_criterion(figure, vecs)
        
        return list(new[i] - self.offset[i] for i in range(len(new)))

    def magnitude_reset_criterion(self, magnitude: float, history: np.array) -> None:
        next_state = self.state
        std = history.std(axis=0).max()
        
        if self.state == OffsetState.AT_REST_ACCUMULATING:
            if magnitude > self.mag_reset_threshold:
                next_state = OffsetState.MOVING
            elif std < 0.2*self.mag_reset_threshold:
                next_state = OffsetState.AT_REST_IDLE
                self.offset = history.mean(axis=0)
                self.state_cb(next_state.name.replace("_", " ") + "!")
                
        elif self.state == OffsetState.AT_REST_IDLE:
            if magnitude > self.mag_reset_threshold:
                next_state = OffsetState.MOVING
                
        elif self.state == OffsetState.MOVING:
            if magnitude < self.mag_reset_threshold:
                next_state = OffsetState.AT_REST_ACCUMULATING

        if next_state != self.state and self.state_cb:
            self.state_cb(next_state.name.replace("_", " "))

        self.state = next_state

In [10]:
class TMeansSubtractor():
    def __init__(self, window: int):
        self.buffer = deque(maxlen=window)

    def __call__(self, new: list[float]) -> list[float]:
        vecs = np.asarray(self.buffer)
        means = np.mean(vecs, axis=0)
        self.buffer.append(new)

        return list(np.asarray(new) - means)


# Live plotting from serial

In [11]:
# Data sampling period [seconds]
sample_period = 10e-3

# Data capture buffer length [amount of samples]
depth = 250

# Live display rate [frames per second]
plot_upd_fps = 30

# data columns constructor
def new_data():
    return dict(
        t=[],
        x=[],
        y=[],
        z=[],
        mag=[],
        dt=[],
    )

In [12]:
source = ColumnDataSource(new_data())
p = figure()
p.line(source=source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=source, x='t', y='z', legend_label="z", line_color="red")
p.line(source=source, x='t', y='mag', legend_label="mag", line_color="black")
p.title.text = "Accelerations"
p.y_range = Range1d(-6, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [13]:
new_samples = new_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
preprocessor = TMeansSubtractor(100)
#preprocessor = deoffset

postprocessor = identity
# postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)


indicator = MotionObserver(1.8, 30,set_title)

# Flush source's data
source.stream(new_samples, 1)
try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        #mag = np.linalg.norm(meas)
        #meas.append(mag)
        meas.append(indicator(meas))
        meas.append(dt/sample_period)

        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(meas[0])
        new_samples['y'].append(meas[1])
        new_samples['z'].append(meas[2])
        new_samples['mag'].append(meas[3])
        new_samples['dt'].append(meas[4])
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = new_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
>>>Change state to SAMPLING


/home/being/.virtualenvs/tests/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/being/.virtualenvs/tests/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Stopped.
All samples are 931


# Velocity

In [14]:

class VelocityTracker:
    def __init__(self, relative_drag: list[float]):
        self.relative_drag = relative_drag
        self.velocity = [0.0,0.0,0.0]

    def __call__(self, new: list[float], dt: float) -> list[float]:
        for i in range(3):
            self.velocity[i] = new[i] * dt + self.velocity[i]
            new[i] = self.velocity[i]
        return new


In [15]:
velocity_source = ColumnDataSource(new_data())
p = figure()
p.line(source=velocity_source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=velocity_source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=velocity_source, x='t', y='z', legend_label="z", line_color="red")
p.line(source=velocity_source, x='t', y='mag', legend_label="mag", line_color="black")
p.title.text = "Accelerations"
p.y_range = Range1d(-6, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [16]:
new_samples = new_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
preprocessor = identity
#preprocessor = deoffset

# postprocessor = identity
postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)

velocity_converter = VelocityTracker([0.5,0.5,0.5])


indicator = MotionObserver(1.8, 30,set_title)

# Flush source's data
velocity_source.stream(new_samples, 1)

current_velocity= {
    'x': 0,
    'y': 0,
    'z': 0
}

try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        meas = velocity_converter(meas,dt)
        #mag = np.linalg.norm(meas)
        #meas.append(mag)
        meas.append(indicator(meas))
        meas.append(dt/sample_period)

        current_velocity['x']

        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(meas[0])
        new_samples['y'].append(meas[1])
        new_samples['z'].append(meas[2])
        new_samples['mag'].append(meas[3])
        new_samples['dt'].append(meas[4])
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            velocity_source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = new_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
>>>Change state to SAMPLING
Stopped.
All samples are 950


# Positional Gesture Detection

In [17]:
test_value = np.array([[1,2,3],[4,5,6], [7,8,9]])
np.linalg.norm(test_value[:,[0,1]], axis=1)

array([ 2.23606798,  6.40312424, 10.63014581])

In [36]:
class Gesture(enum.Enum):
    HORIZONTAL = enum.auto()
    VERTICAL = enum.auto()
    DIAGONAL = enum.auto()


class Analysis():
    def __init__(self, threshold: float = 1.0, diagonal_threshold: float = 0.8, window: int = 20):
        self.buffer = deque(maxlen=window)
        self.threshold = threshold
        self.diagonal_threshold = diagonal_threshold

    def __call__(self, meas: list[float]) -> Optional[Gesture]:
        if len(meas) < 3:
            return None
        self.buffer.append(meas[0:3])
        return self.compute_gesture()

    def horizontal_velocity(self):
        data = np.asarray(self.buffer)
        return np.linalg.norm(data[:,[0,1]], axis=1)

    def vertical_velocity(self):
        data = np.asarray(self.buffer)
        return np.absolute(data[:,2])

    def indicators(self) -> tuple[float, float]:
        horizontal = np.quantile(self.horizontal_velocity(), 0.75)
        vertical = np.quantile(self.vertical_velocity(), 0.75)
        return horizontal, vertical

    def compute_gesture(self, indicators: Optional[tuple[float, float]] = None):
        horizontal, vertical = indicators if indicators else self.indicators()

        if horizontal > self.diagonal_threshold and vertical > self.diagonal_threshold:
            return Gesture.DIAGONAL
        elif horizontal > self.threshold:
            return Gesture.HORIZONTAL
        elif vertical > self.threshold:
            return Gesture.VERTICAL
        else:
            return None

In [56]:
class AnalysisFSM(Analysis):

    DIAGONAL_LOW = math.pi/4 - math.pi/8
    DIAGONAL_HIGH = 1.1*math.pi/4
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.current: Optional[Gesture] = None

    def compute_gesture(self):
        x, y = self.indicators()
        angle = math.atan2(y, x)

        next_state = self.current
        if self.current == None:
            if x > self.diagonal_threshold or y > self.diagonal_threshold:
                if self.DIAGONAL_LOW < angle < self.DIAGONAL_HIGH:
                    next_state = Gesture.DIAGONAL
                elif angle < self.DIAGONAL_LOW:
                    next_state = Gesture.HORIZONTAL
                else:
                    next_state = Gesture.VERTICAL
        else:
            if x < self.diagonal_threshold and y < self.diagonal_threshold:
                next_state = None

        self.current = next_state
        return next_state

In [57]:
def gesture_data():
    return dict(
        t= [],
        x= [],
        y= [],
        x_75percent= [],
        y_75percent= [],
        ratio= [],
    )
gesture_source = ColumnDataSource(gesture_data())
p = figure()
p.line(source=gesture_source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=gesture_source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=gesture_source, x='t', y='x_75percent', legend_label="x", line_color="lightgreen")
p.line(source=gesture_source, x='t', y='y_75percent', legend_label="y", line_color="lightblue")
p.line(source=gesture_source, x='t', y='ratio', legend_label="ratio", line_color="orange")
p.title.text = "Motion"
p.y_range = Range1d(-0.5, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [58]:
import math

new_samples = gesture_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
# preprocessor = identity
#preprocessor = deoffset
preprocessor = TMeansSubtractor(100)

postprocessor = identity
# postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)

indicator = MotionObserver(1.8, 50, None)

analysis = AnalysisFSM(2.0, diagonal_threshold= 1.5, window = 30)
prev_state = None

# Flush source's data
gesture_source.stream(new_samples, 1)
try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        an = analysis(meas)
        if an != prev_state:
            set_title(str(an))
            prev_state = an

        meas.append(indicator(meas))
        meas.append(dt/sample_period)
        
        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(np.mean(analysis.horizontal_velocity()))
        new_samples['y'].append(np.mean(analysis.vertical_velocity()))

        x_75p, y_75p = analysis.indicators()
        ratio = math.atan2(y_75p, x_75p)
        new_samples['x_75percent'].append(x_75p)
        new_samples['y_75percent'].append(y_75p)
        new_samples['ratio'].append(ratio)
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            gesture_source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = gesture_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
>>>Change state to SAMPLING
Stopped.
All samples are 2322


# Offline data analysis

In [ ]:
all_array = np.array(all_samples)

t = np.r_[:all_array.shape[0]] * sample_period
t

p = figure()
p.line(t, all_array[:, 0], legend_label="x", line_color="green")
p.line(t, all_array[:, 1], legend_label="y", line_color="blue")
p.line(t, all_array[:, 2], legend_label="z", line_color="red")
p.line(t, all_array[:, 3], legend_label="mag", line_color="black")

show(p)

### Some tests

In [ ]:
off_ax, off_ay, off_az = all_array.mean(axis=0)[:3]
(off_ax, off_ay, off_az)